In [ ]:
from itertools import chain
from pathlib import Path

from utils.dataset_utils import (
    get_leaning_datasets_for_leave_one_out_benchmark,
    get_leave_one_out_datasets,
)
from utils.model_utils import finetune_models


In [ ]:
TRAINING_POLITICAL_LEANING = True
GET_DATASETS = lambda: chain(
    get_leaning_datasets_for_leave_one_out_benchmark()
)
TEST_DATASET_SAMPLE_FRACTION = 0.15
EVAL_DATASET_SAMPLE_SIZE = 1_000
TRAIN_DATASET_SAMPLE_SIZE = 10_000

BALANCE_CENTER_LEANING_CLASS = False
CENTER_LEANING_CLASS_TRAIN_SIZE_MULTIPLIERS = {
    "article_bias_prediction": 5,
    "bignewsbln": 10,  # There is not enough center leaning examples to balance this out.
    "commoncrawl_news_articles": 5.5,
    "gpt4_political_bias": 3.25,
    "gpt4_political_ideologies": 3,
    "qbias": 3.25,
}
CENTER_LEANING_CLASS_TRAIN_SIZE_MULTIPLIER_DEFAULT = 2.6

train_datasets, eval_datasets = get_leave_one_out_datasets(
    list(GET_DATASETS()),
    TRAINING_POLITICAL_LEANING,
    TEST_DATASET_SAMPLE_FRACTION,
    EVAL_DATASET_SAMPLE_SIZE,
    TRAIN_DATASET_SAMPLE_SIZE,
    BALANCE_CENTER_LEANING_CLASS,
    CENTER_LEANING_CLASS_TRAIN_SIZE_MULTIPLIERS,
    CENTER_LEANING_CLASS_TRAIN_SIZE_MULTIPLIER_DEFAULT,
)


In [ ]:
finetune_models(
    Path(
        "dataset_benchmark",
        "leave_one_out",
        "political_leaning" if TRAINING_POLITICAL_LEANING else "politicalness",
    ),
    train_datasets,
    eval_datasets,
)
